In [1]:
 pip install -q tensorflow-model-optimization

     |████████████████████████████████| 213 kB 6.6 MB/s 


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import math
import tempfile
import zipfile
import os
import matplotlib.pyplot as plt
import seaborn as sns
import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau


# Modifying Data

In [3]:
!ls 
!unzip sign.zip

sample_data  sign.zip
Archive:  sign.zip
  inflating: amer_sign2.png          
  inflating: amer_sign3.png          
  inflating: american_sign_language.PNG  
  inflating: sign_mnist_test.csv     
  inflating: sign_mnist_test/sign_mnist_test.csv  
  inflating: sign_mnist_train.csv    
  inflating: sign_mnist_train/sign_mnist_train.csv  


In [4]:
train_df = pd.read_csv("sign_mnist_train/sign_mnist_train.csv")
test_df = pd.read_csv("sign_mnist_test/sign_mnist_test.csv")
# print(test_df)

In [5]:
test = pd.read_csv("sign_mnist_test/sign_mnist_test.csv")
# print(test)
y = test['label']
# print(y.max())

In [6]:
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,156,158,160,163,165,159,166,168,170,170,171,171,171,172,171,171,170,170,169,111,121,129,135,141,144,148,151,154,157,160,...,205,206,206,207,207,206,206,204,205,204,203,202,142,151,160,172,196,188,188,190,135,96,86,77,77,79,176,205,207,207,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,157,158,156,154,154,153,152,151,149,149,148,147,146,144,142,143,138,92,108,158,159,159,159,160,160,160,160,160,160,160,...,100,78,120,157,168,107,99,121,133,97,95,120,135,116,95,79,69,86,139,173,200,185,175,198,124,118,94,140,133,84,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,186,185,185,185,184,184,184,181,181,179,179,179,178,178,109,52,66,77,83,188,189,189,188,188,189,188,188,188,188,187,...,203,204,203,201,200,200,199,198,196,195,194,193,198,166,132,114,89,74,79,77,74,78,132,188,210,209,206,205,204,203,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,211,209,207,208,207,206,203,202,201,200,198,197,195,192,197,171,51,52,54,212,213,215,215,212,212,213,212,212,211,211,...,247,242,233,231,230,229,227,225,223,221,220,216,58,51,49,50,57,60,17,15,18,17,19,1,159,255,237,239,237,236,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,186,188,189,189,190,191,189,190,190,187,190,192,193,191,191,192,192,194,194,166,169,172,174,177,180,182,185,186,187,190,...,90,77,88,117,123,127,129,134,145,152,156,179,105,106,105,104,104,104,175,199,178,152,136,130,136,150,118,92,85,76,92,105,105,108,133,163,157,163,164,179


In [7]:
train_labels = train_df['label']
test_labels = test_df['label']
del train_df['label']
del test_df['label']

In [8]:
train_images = train_df.values
test_images = test_df.values
# Normalize the data

# print(train_images.shape)
# print(test_images.shape)

train_images = train_images / 255
test_images = test_images / 255
train_images = train_images.reshape(-1,28,28,1)
test_images = test_images.reshape(-1,28,28,1)
# print(train_images.shape)
# print(test_images.shape)

# print(train_df)
# print(train_images)

# Creating CNN model

In [9]:

#setup CNN model
def setup_model():
  model = keras.Sequential([
      # tf.keras.layers.Reshape(input_shape=(28*28,), target_shape(28,28,1)),

      keras.layers.Conv2D(kernel_size = 3, filters = 100, activation = 'relu', padding = 'same', input_shape=(28,28,1)), 
      tf.keras.layers.BatchNormalization(center = True, scale = False),
      tf.keras.layers.Activation('relu'),

      keras.layers.Conv2D(kernel_size = 3, filters = 70, activation = 'relu', padding = 'same', strides = 1),
      tf.keras.layers.BatchNormalization(center = True, scale = False),
      tf.keras.layers.Activation('relu'),
      tf.keras.layers.MaxPool2D((2,2) , strides = 1 , padding = 'same'),
      keras.layers.Dropout(0.25),

      keras.layers.Conv2D(kernel_size = 3, filters = 32, activation = 'relu', padding = 'same', strides = 1),
      tf.keras.layers.BatchNormalization(center = True, scale = False),
      tf.keras.layers.Activation('relu'),



      keras.layers.Flatten(),

      keras.layers.Dense(500, use_bias = False),
      tf.keras.layers.BatchNormalization(center = True, scale = False),
      tf.keras.layers.Activation('relu'),
      keras.layers.Dropout(0.20),

      keras.layers.Dense(100, activation = 'relu'),
      keras.layers.Dropout(0.15),

      keras.layers.Dense(25, activation = 'softmax')
      ])
  return model

In [10]:
#lr decay function
def lr_decay(epoch):
  return 0.01 * math.pow(0.6, epoch)

#lr schedule callback
lr_decay_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay,)

In [11]:
def train_model(model):
  model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])
  model.summary()
  model.fit(train_images, train_labels, epochs=20, callbacks = [lr_decay_callback])
  return model

In [12]:
def save_model_weights(model):
  _, pretrained_weights = tempfile.mkstemp('.h5')
  model.save_weights(pretrained_weights)
  return pretrained_weights

In [13]:
def setup_pretrained_weights():
  model= setup_model()
  model = train_model(model)
  pretrained_weights = save_model_weights(model)
  return pretrained_weights

In [14]:
def save_model_file(model):
  _, keras_file = tempfile.mkstemp('.h5') 
  model.save(keras_file, include_optimizer=False)
  return keras_file

In [15]:
def get_gzipped_model_size(model):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  keras_file = save_model_file(model)

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(keras_file)
  return os.path.getsize(zipped_file)

In [16]:
#define the model
model = setup_model()
# Train the model
model = train_model(model)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 100)       1000      
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 100)       300       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 70)        63070     
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 70)        210       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 70)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 28, 70)        0

In [17]:
# Evaluate the base model
_, baseline_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Baseline test accuracy: 0.9560791850090027
Saving model to:  /tmp/tmp58sfbpyx.h5


# Converting model to tflite

In [18]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model as tflite file.
with open('sign_mnist.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp5tzqxm8h/assets


In [19]:
!ls

american_sign_language.PNG  sample_data		 sign_mnist.tflite     sign.zip
amer_sign2.png		    sign_mnist_test	 sign_mnist_train
amer_sign3.png		    sign_mnist_test.csv  sign_mnist_train.csv


In [20]:
def eval_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [21]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

test_accuracy = eval_model(interpreter)

print('TFLite test_accuracy:', test_accuracy)
# print('Clustered TF test accuracy:', clustered_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.


TFLite test_accuracy: 0.9560791968767429


# Post training dynamic quantization

In [22]:
# Convert model to dynamic range quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

# Save the model as quantized tflite file.
with open('sign_mnist_quant_dyn.tflite', 'wb') as f:
  f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpn2rv4t6v/assets


INFO:tensorflow:Assets written to: /tmp/tmpn2rv4t6v/assets


In [23]:
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

test_accuracy = eval_model(interpreter)

print('Quantized TFLite test_accuracy:', test_accuracy)
# print('Clustered TF test accuracy:', clustered_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.


Quantized TFLite test_accuracy: 0.9559397657557167


# Test the size of the files

In [24]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [25]:
!ls -s sign_mnist.tflite
!ls -s sign_mnist_quant_dyn.tflite

# print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(model)))
# print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(tflite_model)))
# print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size(tflite_quant_model)))

49548 sign_mnist.tflite
12400 sign_mnist_quant_dyn.tflite
